In [4]:
import pandas as pd
import geojson
from geojson import Feature, Point, FeatureCollection

# 读取房屋和车站的 CSV 文件
houses_df = pd.read_csv('..data/curated/property/property 2.csv') 
stations_df = pd.read_csv('../data/raw/external/victoria_metro_stations.csv')  # 替换为你的车站CSV文件路径

houses_df = houses_df.dropna(subset=['longitude', 'latitude'])
stations_df = stations_df.dropna(subset=['longitude', 'latitude'])

# 创建 GeoJSON Feature 的函数
def create_geojson_feature(row, feature_type='house'):
    point = Point((row['longitude'], row['latitude']))
    properties = {
        'address': row.get('address', 'unknown'),  # 如果没有 'address' 列，请根据你的实际列修改
        'type': feature_type
    }
    return Feature(geometry=point, properties=properties)

# 转换房屋数据为 GeoJSON Feature
house_features = houses_df.apply(lambda row: create_geojson_feature(row, 'house'), axis=1).tolist()
house_feature_collection = FeatureCollection(house_features)

# 转换车站数据为 GeoJSON Feature
station_features = stations_df.apply(lambda row: create_geojson_feature(row, 'station'), axis=1).tolist()
station_feature_collection = FeatureCollection(station_features)


# 保存房屋和车站分别为 GeoJSON 文件
houses_geojson_output_path = '/Users/fenglingyi/Downloads/groupwork_generate/property_geojson.geojson'
stations_geojson_output_path = '/Users/fenglingyi/Downloads/groupwork_generate/vic_metro_stations_geojson.geojson'

with open(houses_geojson_output_path, 'w') as f:
    geojson.dump(house_feature_collection, f)

with open(stations_geojson_output_path, 'w') as f:
    geojson.dump(station_feature_collection, f)


print("房屋和车站的GeoJSON文件已生成")

房屋和车站的GeoJSON文件已生成


In [6]:
import geojson
import openrouteservice
import pandas as pd
import time

# 初始化 OpenRouteService API 客户端
api_key = '5b3ce3597851110001cf6248dda07331ae64496fa25c4e54b59fba30'  # 请替换为你的 OpenRouteService API Key
client = openrouteservice.Client(key=api_key)

# 读取 GeoJSON 文件
with open('/Users/fenglingyi/Downloads/groupwork_generate/property_geojson.geojson', 'r') as f:
    rent_geojson = geojson.load(f)

with open('/Users/fenglingyi/Downloads/groupwork_generate/vic_metro_stations_geojson.geojson', 'r') as f:
    stations_geojson = geojson.load(f)

# 提取经纬度
def extract_coordinates(features):
    coordinates = []
    for feature in features['features']:
        lon, lat = feature['geometry']['coordinates']
        address = feature['properties'].get('address', 'unknown')
        coordinates.append((lat, lon, address))
    return coordinates

rent_coordinates = extract_coordinates(rent_geojson)
station_coordinates = extract_coordinates(stations_geojson)

# 提取房屋和车站的经纬度
house_coords = [(lon, lat) for lat, lon, _ in rent_coordinates]
station_coords = [(lon, lat) for lat, lon, _ in station_coordinates]

# 使用 OpenRouteService API 批量计算距离并按批次处理
def calculate_distance_matrix_in_batches(house_coords, station_coords, batch_size=12):
    all_shortest_distances = []
    
    for i in range(0, len(house_coords), batch_size):
        batch_houses = house_coords[i:i + batch_size]
        try:
            matrix = client.distance_matrix(
                locations=batch_houses + station_coords,
                profile='driving-car',
                sources=list(range(len(batch_houses))),
                destinations=list(range(len(batch_houses), len(batch_houses) + len(station_coords))),
                metrics=['distance'],
                units='km'
            )
            distances = matrix['distances']
            
            # 处理每个房屋和车站的距离，找出最短的距离
            for j, house_distances in enumerate(distances[:len(batch_houses)]):
                # 排除 None 值的距离，找出最小距离
                valid_distances = [d for d in house_distances if d is not None]
                if valid_distances:
                    min_distance = min(valid_distances)
                    closest_station_idx = house_distances.index(min_distance)
                    rent_address = rent_coordinates[i + j][2]
                    closest_station = station_coordinates[closest_station_idx][2]
                    all_shortest_distances.append({
                        'rent_address': rent_address,
                        'closest_station': closest_station,
                        'minimum_distance_station': min_distance
                    })
        except Exception as e:
            print(f"Error calculating distance matrix: {e}")
        time.sleep(1)  # 防止触发速率限制
    
    return all_shortest_distances

# 分批处理
shortest_distances = calculate_distance_matrix_in_batches(house_coords, station_coords, batch_size=12)

# 保存最短距离结果为 CSV 文件
shortest_distances_df = pd.DataFrame(shortest_distances)
output_path = '/Users/fenglingyi/Downloads/groupwork_final/closest_stations_distance_final_2.csv'
shortest_distances_df.to_csv(output_path, index=False)

print(f"结果已保存到 {output_path}")


结果已保存到 /home/yanbinh/data/rent_closest_station_distance_matrix_batched.csv
